In [68]:
import xmltodict, json
from os import listdir
from os.path import isfile, join
import re

In [37]:
#READ THE GOODNESS
queries = open("testRunOut/query", "r").read()
queries_d = xmltodict.parse(queries)

bugs = open("SWTBugRepository.xml", "r").read()
bugs_d = xmltodict.parse(bugs)

docs_path = "testRunOut/docs/"
docs_files = [f for f in listdir(docs_path) if isfile(join(docs_path, f))]
docs_str_d = { d : open("testRunOut/docs/"+d, "r").read() for d in docs_files }
docs_d = { d : xmltodict.parse(docs_str_d[d]) for d in docs_files }

In [107]:
#CHECK THOSE TURDS
#json.dumps(bugs_d)
#json.dumps(queries_d)
#json.dumps(docs_d)

In [142]:
#CLEAN THAT SHITE
bug_map = {}
for b in bugs_d['bugrepository']['bug']:
    if type(b['fixedFiles']['file']) is not list:
        file_list = [b['fixedFiles']['file']]
    else:
        file_list = b['fixedFiles']['file']
    bug_map[b['@id']] =  file_list #number -> files
#json.dumps(bug_map)

query_map = {}
c = re.compile(r"(\S+)\..class.")
m = re.compile(r"(\S+)\..method.")
i = re.compile(r"(\S+)\..identifier.") 
o = re.compile(r"(\S+)\..comments.")
for q in queries_d['parameters']['query']:
    text = q['text']
    c_l = list(set(c.findall(text)))
    m_l = list(set(m.findall(text)))
    i_l = list(set(i.findall(text)))
    o_l = list(set(o.findall(text)))
    query_map[q['number']] = {'class':c_l, 'method':m_l, 'identifier':i_l, 'comments':o_l}
#json.dumps(query_map)   

docs_map = {}
for _id in docs_d:
    _file = docs_d[_id]['DOC']['DOCNO']
    
    c = docs_d[_id]['DOC']['text']['class'] #WTF ... org.eclipse.swt.internal.win32.OS.java <class> is empty
    if c is None: c = ''
    
    c_l = list(set(c.split('\n')))
    m_l = list(set(docs_d[_id]['DOC']['text']['method'].split('\n')))
    i_l = list(set(docs_d[_id]['DOC']['text']['identifier'].split('\n')))
    o_l = list(set(docs_d[_id]['DOC']['text']['comments'].split('\n'))) 
    docs_map[_file] = {'class':c_l, 'method':m_l, 'identifier':i_l, 'comments':o_l} #'id':_id,
#json.dumps(docs_map)       


In [146]:
#THE REAL MAPPING BEGINS
query2doc_map = {}
for num in bug_map:
    for _file in bug_map[num]: 
        for field in docs_map[_file]:
            for q_line in query_map[num][field]:
                if field is 'comments': continue
                for d_line in docs_map[_file][field]:
                    if not d_line.strip(): continue #if blank or empty skip
                
                    if q_line in query2doc_map:
                        query2doc_map[q_line].append(d_line)
                    else:
                        query2doc_map[q_line] = [d_line]

for line in query2doc_map:
    temp = list(set(query2doc_map[line]))
    query2doc_map[line] = temp
                


In [147]:
print query_map[num][field]
print docs_map[_file][field]

[u'and', u'another', u'chinese', u'text', u'misc', u'back', u'are', u'have', u'open', u'again', u'ime', u'window', u'editor', u'closed', u'status', u'then', u'IME', u'Eclipse', u'Chinese', u'lost', u'eclipse', u'switch', u'Open']
[u'set enabled', u'set bounds', u' enabled', u' getminmaxinfo', u'set active control', u'widget parent', u' activate', u'get shells', u'enable widget', u' dispose', u'win 32_new', u'set tool tip text', u' syscommand', u' command', u'get enabled', u'set visible', u' querynewpalette', u'set item enabled', u'get shell', u'force active', u'fix shell', u' setcursor', u'force resize', u'get location', u' palettechanged', u' windowposchanging', u'find cursor', u'set region', u' showwindow', u' close', u'set minimum size', u'call window proc', u' open', u' visible', u'widget style', u'update modal', u'shell', u'translate accelerator', u'release shells', u'widget create struct', u'window class', u' settingchange', u'get bounds', u'find theme control', u'traverse escape

In [148]:
with open('query2doc_map.txt', 'w') as outfile:
     json.dump(query2doc_map, outfile, sort_keys = True, indent = 4, ensure_ascii=False)